In [123]:
# Optical flow tracks objects by looking at where the same points have moved from one image frame to the next.
# Let's load in a few example frames of a pacman-like face moving to the right and down and see how optical flow 
# finds motion vectors that describe the motion of the face!

In [4]:
import numpy as np
import os
import matplotlib.image as mpimg  # for reading in images
import matplotlib.pyplot as plt
import cv2

%matplotlib inline

In [5]:
def load_images_from_folder(folder):
    images = []
    for filename in np.sort(os.listdir(folder)):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

In [ ]:
frames = load_images_from_folder('images/')
np.shape(frames)


In [ ]:
frames=[cv2.cvtColor(frames[i], cv2.COLOR_BGR2RGB) for i in range(0,np.shape(frames)[0])]
np.shape(frames)

In [ ]:
fig,a = plt.subplots(1, np.shape(frames)[0], figsize=(20,20))
for i in range (0,np.shape(frames)[0]):
    a[i].set_title('frame '+ str(i))
    a[i].imshow(frames[i])

In [ ]:
# convert all frames to grayscale
# gray = 

In [10]:
gray_1=gray[0]
gray_2=gray[1]
gray_3=gray[2]
gray_4=gray[3]
gray_5=gray[4]
gray_6=gray[5]

In [ ]:
# Finding Points to Track
# Before optical flow can work, we have to give it a set of keypoints to track between two image frames!

# In the below example, we use a Shi-Tomasi corner detector, which uses the same process as a Harris corner detector 
# to find patterns of intensity that make up a "corner" in an image, only it adds an additional parameter that
# helps select the most prominent corners. You can read more about this detection algorithm in the documentation.

# Alternatively, the Harris detector could be used to find feature points.


In [ ]:
# parameters for ShiTomasi corner detection
feature_params = dict( maxCorners = 1,
                       qualityLevel = 0.2,
                       minDistance = 150,
                       blockSize = 5 )


# Take first frame and find corner points in it
pts_1 = cv2.goodFeaturesToTrack(gray_1, mask = None, **feature_params)

# display the detected points
plt.imshow(frames[0])
for p in pts_1:
    # plot x and y detected points
    plt.plot(p[0][0], p[0][1], 'r.', markersize=15)

# print out the x-y locations of the detected points
print(pts_1)


In [ ]:
# Perform Optical Flow

# Once we've detected keypoints on our initial image of interest, we can calculate
#the optical flow between this image frame (frame 1) and the next frame (frame 2),
# using OpenCV's calcOpticalFlowPyrLK which is documented, here. It takes in an initial image frame,
# the next image, and the first set of points, and it returns the detected points in the next frame
# and a value that indicates how good matches are between points from one frame to the next.

# The parameters also include a window size and maxLevels that indicate the size of a window and mnumber 
# of levels that will be used to scale the given images using pyramid scaling; this version peforms an iterative
# search for matching points and this matching criteria is reflected in the last parameter 
# (you may need to change these values if you are working with a different image,
#but these should work for the provided example).

In [ ]:
# parameters for lucas kanade optical flow
lk_params = dict( winSize  = (1,1),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Take first frame and find corner points in it
pts_2 = cv2.goodFeaturesToTrack(gray_2, mask = None, **feature_params)

plt.imshow(frames[1])
for p in pts_2:
    # plot x and y detected points
    plt.plot(p[0][0], p[0][1], 'r.', markersize=15)

# print out the x-y locations of the detected points
print(pts_2)


In [ ]:
# calculate optical flow between first and second frame
pts_2, match, err = cv2.calcOpticalFlowPyrLK(gray_1, gray_2, pts_1,None, **lk_params)

# Select good matching points between the two image frames
good_new = pts_2[match==1]
good_old = pts_1[match==1]
print(pts_2)

In [ ]:
print(match)

In [ ]:
# Next, let's display the resulting motion vectors! You should see the first image with motion vectors drawn
# on it that indicate the direction of motion from the first frame to the next.


In [ ]:
mask = np.zeros_like(frames[1])

# draw the lines between the matching points (these lines indicate motion vectors)
for i,(new,old) in enumerate(zip(good_new,good_old)):
    a,b = new.ravel()
    c,d = old.ravel()
    a = int(a)
    b = int(b)
    c = int(c)
    d = int(d)
    # draw points on the mask image
    mask = cv2.circle(mask,(a,b),5,(200),-1)
    # draw motion vector as lines on the mask image
    mask = cv2.line(mask, (a,b),(c,d), (200), 3)
    # add the line image and second frame together

composite_im = np.copy(frames[1])
composite_im[mask!=0] = [0]

plt.imshow(composite_im)

In [ ]:
## TODO: Perform optical flow between image frames 2 and 3

# Now take the second frame and find corner points in it just like first frame
pts_2 = cv2.goodFeaturesToTrack(gray_2, mask = None, **feature_params)

# calculate second and third frame
pts_3, match, err = cv2.calcOpticalFlowPyrLK(gray_2, gray_3, pts_2, None, **lk_params)

# good matching points between the two frames
good_new = pts_3[match==1]
good_old = pts_2[match==1]

# mask image for drawing (u,v) vectors on top of the third frame
mask = np.zeros_like(frames[2])

# this will draw lines between the matching points (lines indicate motion)
for i,(new,old) in enumerate(zip(good_new,good_old)):
    a,b = new.ravel()
    c,d = old.ravel()
    a = int(a)
    b = int(b)
    c = int(c)
    d = int(d)
    # draw points on the mask image
    mask = cv2.circle(mask,(a,b),5,(200),-1)
    # draw motion vector as lines on the mask image
    mask = cv2.line(mask, (a,b),(c,d), (200), 3)
    # add the line image and second frame together

composite_im = np.copy(frames[2])
composite_im[mask!=0] = [0]

plt.imshow(composite_im)

Homework

(Mandatory)
- Apply lk optical flow on all frames
- Draw the full feature tracking history over all frames
- Use different sequence of images

(Optional)
- Use dense optical flow (Farneback) on the same images